In [ ]:
!pip install tensorflow_addons -q

     |████████████████████████████████| 1.1 MB 15.4 MB/s 


In [ ]:
!pip install comet_ml -q

     |████████████████████████████████| 441 kB 15.8 MB/s 
     |████████████████████████████████| 130 kB 66.9 MB/s 
     |████████████████████████████████| 54 kB 2.6 MB/s 
     |████████████████████████████████| 54 kB 3.1 MB/s 
     |████████████████████████████████| 498 kB 58.6 MB/s 
     |████████████████████████████████| 168 kB 63.3 MB/s 
     |████████████████████████████████| 140 kB 73.5 MB/s 
     |████████████████████████████████| 127 kB 61.6 MB/s 
     |████████████████████████████████| 166 kB 75.3 MB/s 
     |████████████████████████████████| 166 kB 72.6 MB/s 
     |████████████████████████████████| 162 kB 74.1 MB/s 
     |████████████████████████████████| 162 kB 73.2 MB/s 
     |████████████████████████████████| 158 kB 72.7 MB/s 
     |████████████████████████████████| 157 kB 73.3 MB/s 
     |████████████████████████████████| 157 kB 78.5 MB/s 
     |████████████████████████████████| 157 kB 78.2 MB/s 
     |████████████████████████████████| 157 kB 70.7 MB/s 
     |████████████

In [ ]:
!pip install python-dotenv -q

In [ ]:
!pip install keras-tuner --upgrade -q

     |████████████████████████████████| 135 kB 15.5 MB/s 
     |████████████████████████████████| 1.6 MB 59.9 MB/s 


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from dotenv import load_dotenv
load_dotenv();

import os

In [ ]:
# import comet_ml at the top of your file
from comet_ml import Experiment

# Create an experiment with your api key
experiment = Experiment(
    api_key=os.environ.get('COMET_API_KEY'),
    project_name="mlp-keras-2",
    workspace="ift6758-a22-g08",
    auto_param_logging=True,
    auto_histogram_weight_logging=True,
    auto_histogram_gradient_logging=True,
    auto_histogram_activation_logging=True,)

COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET ERROR: Failed to calculate active processors count. Fall back to default CPU count 1
COMET INFO: Couldn't find a Git repository in '/content' nor in any parent directory. You can override where Comet is looking for a Git Patch by setting the configuration `COMET_GIT_DIRECTORY`
COMET INFO: Experiment is live on comet.com https://www.comet.com/ift6758-a22-g08/mlp-keras-2/f2e02aca5e7b40a08ed0fcdaa8b7eb86



In [ ]:
import pandas as pd
# import keras_tuner
from tensorflow.keras import Input, Model, models, layers, losses, metrics, optimizers, callbacks, regularizers
from keras.models import Sequential
from sklearn.model_selection import train_test_split
import tensorflow_addons as tfa

In [ ]:
X_train = pd.read_csv('/content/drive/MyDrive/NHL/X_train.csv')
X_val = pd.read_csv('/content/drive/MyDrive/NHL/X_valid.csv')
y_train = pd.read_csv('/content/drive/MyDrive/NHL/y_train.csv').squeeze()
y_val = pd.read_csv('/content/drive/MyDrive/NHL/y_valid.csv').squeeze()

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)
X_val =  scaler.transform(X_val)

In [ ]:
X_train.shape, y_train.shape, X_val.shape, y_val.shape

((244051, 37), (244051,), (61008, 37), (61008,))

In [ ]:
from sklearn.utils import class_weight
import numpy as np

class_weights = class_weight.compute_class_weight('balanced',
                                                 classes=np.unique(y_train),
                                                 y=y_train)
class_weights = dict(zip(np.unique(y_train), class_weights))
class_weights

{0.0: 0.5503163657846909, 1.0: 5.468562337545936}

In [ ]:
f1 = tfa.metrics.F1Score(num_classes=1, threshold=0.5,  average='macro')

In [ ]:
# baseline model
def build_model():
	# create model
  x = Input(shape=(37,))
  l2 = layers.Dense(64, activation='relu', kernel_regularizer=regularizers.L1L2(l1=1e-5, l2=1e-4))(x)
  l3 = layers.Dense(32, activation='relu', kernel_regularizer=regularizers.L1L2(l1=1e-5, l2=1e-4))(l2)
  l4 = layers.Dense(16, activation='relu', kernel_regularizer=regularizers.L1L2(l1=1e-5, l2=1e-4))(l3) 
  logits = layers.Dense(1, activation='sigmoid')(l4)
  
  model = Model(inputs=x, outputs=logits)
  lr_schedule = optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-4,
    decay_steps=10000,
    decay_rate=0.9)
  model.compile(optimizer=optimizers.Adam(
    learning_rate=lr_schedule), loss='binary_crossentropy', metrics=['acc', f1])
  
  return model
model = build_model()

In [ ]:
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 37)]              0         
                                                                 
 dense_4 (Dense)             (None, 64)                2432      
                                                                 
 dense_5 (Dense)             (None, 32)                2080      
                                                                 
 dense_6 (Dense)             (None, 16)                528       
                                                                 
 dense_7 (Dense)             (None, 1)                 17        
                                                                 
Total params: 5,057
Trainable params: 5,057
Non-trainable params: 0
_________________________________________________________________


In [ ]:
callbacks_list = [callbacks.ModelCheckpoint(filepath='/content/drive/MyDrive/NHL/MLP_Keras_3.h5', monitor='val_f1_score', mode='max', save_best_only=True,)]

In [ ]:
batch_size = 512
num_classes = 1
epochs = 100
num_nodes = 64
optimizer = 'adam'
activation = 'relu'

params={'batch_size':batch_size,
        'epochs':epochs,
        'layer1_type':'Dense',
        'layer1_num_nodes':num_nodes,
        'layer1_activation':activation,
        'optimizer':optimizer
}

In [ ]:
history = model.fit(x=X_train, 
                    y=y_train, 
                    validation_data=(X_val, y_val), 
                    class_weight=class_weights,
                    batch_size=batch_size, 
                    epochs=epochs, 
                    verbose=1,
                    callbacks=callbacks_list)

COMET INFO: Ignoring automatic log_parameter('verbose') because 'keras:verbose' is in COMET_LOGGING_PARAMETERS_IGNORE


Epoch 1/100
477/477 [==============================] - 5s 8ms/step - loss: 0.6627 - acc: 0.4891 - f1_score: 0.2150 - val_loss: 0.6286 - val_acc: 0.6574 - val_f1_score: 0.2676
Epoch 2/100
477/477 [==============================] - 3s 6ms/step - loss: 0.6020 - acc: 0.6733 - f1_score: 0.2790 - val_loss: 0.5895 - val_acc: 0.6789 - val_f1_score: 0.2873
Epoch 3/100
477/477 [==============================] - 2s 5ms/step - loss: 0.5836 - acc: 0.6808 - f1_score: 0.2908 - val_loss: 0.5749 - val_acc: 0.6831 - val_f1_score: 0.2945
Epoch 4/100
477/477 [==============================] - 3s 6ms/step - loss: 0.5741 - acc: 0.6831 - f1_score: 0.2960 - val_loss: 0.5740 - val_acc: 0.6778 - val_f1_score: 0.2948
Epoch 5/100
477/477 [==============================] - 2s 5ms/step - loss: 0.5682 - acc: 0.6808 - f1_score: 0.2977 - val_loss: 0.5574 - val_acc: 0.6874 - val_f1_score: 0.2992
Epoch 6/100
477/477 [==============================] - 3s 6ms/step - loss: 0.5641 - acc: 0.6841 - f1_score: 0.3006 - val_loss

In [ ]:
experiment.log_dataset_hash(X_train)
experiment.log_model("MLP_keras", '/content/drive/MyDrive/NHL/MLP_Keras_3.h5')

{'web': 'https://www.comet.com/api/asset/download?assetId=3f8b2f67a1de4d7e98f3162a0b2b0bc3&experimentKey=f2e02aca5e7b40a08ed0fcdaa8b7eb86',
 'api': 'https://www.comet.com/api/rest/v2/experiment/asset/get-asset?assetId=3f8b2f67a1de4d7e98f3162a0b2b0bc3&experimentKey=f2e02aca5e7b40a08ed0fcdaa8b7eb86',
 'assetId': '3f8b2f67a1de4d7e98f3162a0b2b0bc3'}

In [ ]:
experiment.end()

COMET INFO: ---------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.com/ift6758-a22-g08/mlp-keras-2/f2e02aca5e7b40a08ed0fcdaa8b7eb86
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     acc [100]                      : (0.48911088705062866, 0.7073603272438049)
COMET INFO:     batch_acc [4800]               : (0.107421875, 0.748046875)
COMET INFO:     batch_f1_score [4800]          : (0.15837937593460083, 0.42276424169540405)
COMET INFO:     batch_loss [4800]              : (0.41878563165664673, 0.7489876747131348)
COMET INFO:     epoch_duration [100]           : (2.2441442729999608, 6.7550423699999556)
COMET INFO:     f1_score [100]                 : (0.21495626866817474, 0.3226270377635956)
COMET INFO:     loss [100]                     : (0.5294990539550781, 0.6626832485198975)
COMET INFO:     val_acc [100]   

In [ ]:
# import comet_ml at the top of your file
from comet_ml import Experiment

# Create an experiment with your api key
experiment = Experiment(
    api_key=os.environ.get('COMET_API_KEY'),
    project_name="mlp-keras-2",
    workspace="ift6758-a22-g08",
    auto_param_logging=True,
    auto_histogram_weight_logging=True,
    auto_histogram_gradient_logging=True,
    auto_histogram_activation_logging=True,)

COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET ERROR: Failed to calculate active processors count. Fall back to default CPU count 1
COMET INFO: Couldn't find a Git repository in '/content' nor in any parent directory. You can override where Comet is looking for a Git Patch by setting the configuration `COMET_GIT_DIRECTORY`
COMET INFO: Experiment is live on comet.com https://www.comet.com/ift6758-a22-g08/mlp-keras-2/bc8831be0a7949f2b41aae9d325e3269



In [ ]:
loaded_model = models.load_model('/content/drive/MyDrive/NHL/MLP_Keras_3.h5')

In [ ]:
callbacks_list = [callbacks.ModelCheckpoint(filepath='/content/drive/MyDrive/NHL/MLP_Keras_3.h5', monitor='val_f1_score', mode='max', save_best_only=True,)]

In [ ]:
batch_size = 512
num_classes = 1
epochs = 200
num_nodes = 64
optimizer = 'adam'
activation = 'relu'

params={'batch_size':batch_size,
        'epochs':epochs,
        'layer1_type':'Dense',
        'layer1_num_nodes':num_nodes,
        'layer1_activation':activation,
        'optimizer':optimizer
}

In [ ]:
history2 = loaded_model.fit(x=X_train, 
                    y=y_train, 
                    validation_data=(X_val, y_val), 
                    class_weight=class_weights,
                    batch_size=batch_size, 
                    epochs=epochs, 
                    verbose=1,
                    callbacks=callbacks_list)

Epoch 1/200
477/477 [==============================] - 4s 6ms/step - loss: 0.5353 - acc: 0.7018 - f1_score: 0.3178 - val_loss: 0.5416 - val_acc: 0.6964 - val_f1_score: 0.3105
Epoch 2/200
477/477 [==============================] - 3s 6ms/step - loss: 0.5351 - acc: 0.7007 - f1_score: 0.3175 - val_loss: 0.5317 - val_acc: 0.7034 - val_f1_score: 0.3126
Epoch 3/200
477/477 [==============================] - 2s 5ms/step - loss: 0.5349 - acc: 0.7016 - f1_score: 0.3179 - val_loss: 0.5342 - val_acc: 0.7014 - val_f1_score: 0.3120
Epoch 4/200
477/477 [==============================] - 2s 5ms/step - loss: 0.5347 - acc: 0.7021 - f1_score: 0.3182 - val_loss: 0.5388 - val_acc: 0.6978 - val_f1_score: 0.3107
Epoch 5/200
477/477 [==============================] - 2s 5ms/step - loss: 0.5346 - acc: 0.7011 - f1_score: 0.3180 - val_loss: 0.5452 - val_acc: 0.6932 - val_f1_score: 0.3090
Epoch 6/200
477/477 [==============================] - 2s 5ms/step - loss: 0.5343 - acc: 0.7019 - f1_score: 0.3186 - val_loss